In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, trim, col,concat_ws, udf
from pyspark.ml.feature import Tokenizer

In [2]:
spark = SparkSession.builder.appName('EndSem3').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/26 20:07:57 WARN Utils: Your hostname, Karthikeya, resolves to a loopback address: 127.0.1.1; using 172.25.191.235 instead (on interface zt4homnczt)
25/10/26 20:07:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/26 20:07:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df1 = spark.read.csv('file:///home/karthikeya/Desktop/sem5/MIT_SEM5_BDA/Week3/datasets/lab3_1_dataset.csv', header=True, inferSchema=True)
df2 = spark.read.csv('file:///home/karthikeya/Desktop/sem5/MIT_SEM5_BDA/Week3/datasets/lab3_2_dataset.csv', header=True, inferSchema=True)

In [9]:
df = df1.union(df2)

In [10]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)



In [11]:
df.show(5)

+----+--------------+--------------------+
|  id|          name|             address|
+----+--------------+--------------------+
|1859|    Nkev Koquo|1087 Uxkgk St, Zw...|
|1402| Awdeltx Flswj|1313 Bysqtepcyb S...|
|3503| Zvgk Qmnjjfub|4451 Uymmthnzp St...|
|1009|Suaxud Ybukuxq|4793 Fodikswks St...|
|2737|          NULL|                NULL|
+----+--------------+--------------------+
only showing top 5 rows


In [14]:
df = df.withColumn('name', lower(trim(col('name'))))
df = df.withColumn('address', lower(trim(col('address'))))

In [16]:
df = df.na.drop()

In [19]:
df = df.withColumn('tokems_str', concat_ws(' ', col('name'), col('address')))

In [21]:
toeknizer = Tokenizer(inputCol='tokems_str', outputCol='tokens')
df = toeknizer.transform(df)

In [22]:
df.show(5)

+----+---------------+--------------------+--------------------+--------------------+
|  id|           name|             address|          tokems_str|              tokens|
+----+---------------+--------------------+--------------------+--------------------+
|1859|     nkev koquo|1087 uxkgk st, zw...|nkev koquo 1087 u...|[nkev, koquo, 108...|
|1402|  awdeltx flswj|1313 bysqtepcyb s...|awdeltx flswj 131...|[awdeltx, flswj, ...|
|3503|  zvgk qmnjjfub|4451 uymmthnzp st...|zvgk qmnjjfub 445...|[zvgk, qmnjjfub, ...|
|1009| suaxud ybukuxq|4793 fodikswks st...|suaxud ybukuxq 47...|[suaxud, ybukuxq,...|
| 276|bdiolpnl iduwpc|3640 hrfulh st, c...|bdiolpnl iduwpc 3...|[bdiolpnl, iduwpc...|
+----+---------------+--------------------+--------------------+--------------------+
only showing top 5 rows


In [29]:
def similarity_jaccard(tokens1, tokens2):
    tokens1 = set(tokens1)
    tokens2 = set(tokens2)

    unio = len(tokens1.union(tokens2))
    inter = len(tokens2.intersection(tokens2))

    if not unio:
        return 0.0
    return inter / unio

similarity_jaccard_udf = udf(similarity_jaccard)

In [33]:
df_pairs = df.alias('a').crossJoin(df.alias('b')).filter('a.id != b.id')

In [34]:
df_pairs = df_pairs.withColumn('similarity_jaccard', similarity_jaccard_udf(col('a.tokens'), col('b.tokens')))

In [35]:
df_pairs.show(5)

+----+----------+--------------------+--------------------+--------------------+----+---------------+--------------------+--------------------+--------------------+------------------+
|  id|      name|             address|          tokems_str|              tokens|  id|           name|             address|          tokems_str|              tokens|similarity_jaccard|
+----+----------+--------------------+--------------------+--------------------+----+---------------+--------------------+--------------------+--------------------+------------------+
|1859|nkev koquo|1087 uxkgk st, zw...|nkev koquo 1087 u...|[nkev, koquo, 108...|1402|  awdeltx flswj|1313 bysqtepcyb s...|awdeltx flswj 131...|[awdeltx, flswj, ...|0.5454545454545454|
|1859|nkev koquo|1087 uxkgk st, zw...|nkev koquo 1087 u...|[nkev, koquo, 108...|3503|  zvgk qmnjjfub|4451 uymmthnzp st...|zvgk qmnjjfub 445...|[zvgk, qmnjjfub, ...|0.5454545454545454|
|1859|nkev koquo|1087 uxkgk st, zw...|nkev koquo 1087 u...|[nkev, koquo, 108...|

In [42]:
df_dup = df_pairs.filter('similarity_jaccard > 0.8')

In [43]:
df_dup.show()

+----+----------------+--------------------+--------------------+--------------------+----+----------------+--------------------+--------------------+--------------------+------------------+
|  id|            name|             address|          tokems_str|              tokens|  id|            name|             address|          tokems_str|              tokens|similarity_jaccard|
+----+----------------+--------------------+--------------------+--------------------+----+----------------+--------------------+--------------------+--------------------+------------------+
|8400|    nzoxo mjzsyd|9393 ecswswsif st...|nzoxo mjzsyd 9393...|[nzoxo, mjzsyd, 9...|3400|    nzoxo mjzsyd|9393 ecswswsif st...|nzoxo mjzsyd 9393...|[nzoxo, mjzsyd, 9...|0.8571428571428571|
|9900|    plhcnv qoszc|4388 bravo street...|plhcnv qoszc 4388...|[plhcnv, qoszc, 4...|4900|    plhcnv qoszc|4388 bravo st, np...|plhcnv qoszc 4388...|[plhcnv, qoszc, 4...|0.8571428571428571|
|6200|       wbk jesen|4929 bhlrdo stree...|w

In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [46]:
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [47]:
spark.stop()